In [1]:
import re
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import tensorflow as tf

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping

In [2]:
fake_data = pd.read_csv('C:/Users/Windows10/Desktop/Python/data/fake_news/Fake.csv')
true_data = pd.read_csv('C:/Users/Windows10/Desktop/Python/data/fake_news/True.csv')

In [3]:
fake_data.head()

,title,text,subject,date
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017"
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017"
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017"
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017"
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017"


In [4]:
true_data.head()

,title,text,subject,date
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017"
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017"
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017"
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017"
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017"


In [5]:
fake_data['target'] = 0
true_data['target'] = 1

print(fake_data.head())
print('-'*20)
print(true_data.head())

                                               title  \
0   Donald Trump Sends Out Embarrassing New Year’...   
1   Drunk Bragging Trump Staffer Started Russian ...   
2   Sheriff David Clarke Becomes An Internet Joke...   
3   Trump Is So Obsessed He Even Has Obama’s Name...   
4   Pope Francis Just Called Out Donald Trump Dur...   

                                                text subject  \
0  Donald Trump just couldn t wish all Americans ...    News   
1  House Intelligence Committee Chairman Devin Nu...    News   
2  On Friday, it was revealed that former Milwauk...    News   
3  On Christmas day, Donald Trump announced that ...    News   
4  Pope Francis used his annual Christmas Day mes...    News   

                date  target  
0  December 31, 2017       0  
1  December 31, 2017       0  
2  December 30, 2017       0  
3  December 29, 2017       0  
4  December 25, 2017       0  
--------------------
                                               title  \
0  As U.S. budg

In [6]:
df = pd.concat([true_data, fake_data])
df

,title,text,subject,date,target
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017",1
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017",1
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017",1
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017",1
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017",1
...,...,...,...,...,...
23476,McPain: John McCain Furious That Iran Treated ...,21st Century Wire says As 21WIRE reported earl...,Middle-east,"January 16, 2016",0
23477,JUSTICE? Yahoo Settles E-mail Privacy Class-ac...,21st Century Wire says It s a familiar theme. ...,Middle-east,"January 16, 2016",0
23478,Sunnistan: US and Allied ‘Safe Zone’ Plan to T...,Patrick Henningsen 21st Century WireRemember ...,Middle-east,"January 15, 2016",0
23479,How to Blow $700 Million: Al Jazeera America F...,21st Century Wire says Al Jazeera America will...,Middle-east,"January 14, 2016",0


In [7]:
df = df.drop(['subject', 'date'], axis = 1)
df['text_'] = df['title'] + ' ' + df['text']
df = df.drop(['title', 'text'], axis = 1)

df.head()

,target,text_
0,1,"As U.S. budget fight looms, Republicans flip t..."
1,1,U.S. military to accept transgender recruits o...
2,1,Senior U.S. Republican senator: 'Let Mr. Muell...
3,1,FBI Russia probe helped by Australian diplomat...
4,1,Trump wants Postal Service to charge 'much mor...


In [8]:
df = df[['text_', 'target']]
df.head()

,text_,target
0,"As U.S. budget fight looms, Republicans flip t...",1
1,U.S. military to accept transgender recruits o...,1
2,Senior U.S. Republican senator: 'Let Mr. Muell...,1
3,FBI Russia probe helped by Australian diplomat...,1
4,Trump wants Postal Service to charge 'much mor...,1


In [9]:
df_shuffled = df.sample(frac = 1).reset_index(drop = True)

In [10]:
np.bincount(df_shuffled['target'])

array([23481, 21417], dtype=int64)

In [11]:
train_split = int(len(df_shuffled) * 0.75)

x_train = df_shuffled['text_'][:train_split]
x_test = df_shuffled['text_'][train_split:]

y_train = df_shuffled['target'][:train_split]
y_test = df_shuffled['target'][train_split:]

In [12]:
t = Tokenizer(num_words= 10000, filters = '!"#$%&()*+,-./:;<=>?@[\]^_`{"}~\t\n', lower = True, split = ' ')
t.fit_on_texts(x_train)
x_train_seq = t.texts_to_sequences(x_train)
x_test_seq = t.texts_to_sequences(x_test)


In [31]:
x_train_chunk = pad_sequences(x_train_seq, maxlen = 200)
x_test_chunk = pad_sequences(x_test_seq, maxlen = 200)

In [32]:
model = Sequential([
    Embedding(len(t.word_index) + 1, 32, input_length= 200),
    LSTM(64, activation= 'tanh'),
    Dense(32, activation= 'relu'),
    Dense(16, activation = 'relu'),
    Dense(1, activation = 'sigmoid')
])

In [33]:
model.compile(loss = 'binary_crossentropy', optimizer = 'rmsprop', metrics= ['acc'])
es = EarlyStopping(monitor= 'val_loss', patience= 5)

In [34]:
model.fit(x_train_chunk, y_train, epochs = 200, batch_size = 64, validation_split= 0.2, callbacks= [es])

Epoch 1/200
421/421 [==============================] - 61s 139ms/step - loss: 0.1498 - acc: 0.9532 - val_loss: 0.0413 - val_acc: 0.9890
Epoch 2/200
421/421 [==============================] - 59s 139ms/step - loss: 0.0315 - acc: 0.9908 - val_loss: 0.0355 - val_acc: 0.9889
Epoch 3/200
421/421 [==============================] - 59s 139ms/step - loss: 0.0239 - acc: 0.9937 - val_loss: 0.0373 - val_acc: 0.9898
Epoch 4/200
421/421 [==============================] - 58s 139ms/step - loss: 0.0166 - acc: 0.9957 - val_loss: 0.0353 - val_acc: 0.9893
Epoch 5/200
421/421 [==============================] - 59s 141ms/step - loss: 0.0146 - acc: 0.9963 - val_loss: 0.0294 - val_acc: 0.9923
Epoch 6/200
421/421 [==============================] - 59s 140ms/step - loss: 0.0125 - acc: 0.9967 - val_loss: 0.0290 - val_acc: 0.9921
Epoch 7/200
421/421 [==============================] - 59s 141ms/step - loss: 0.0089 - acc: 0.9978 - val_loss: 0.0353 - val_acc: 0.9933
Epoch 8/200
421/421 [===========================